![Qiskit](https://github.com/Qiskit/qiskit-tutorials/raw/115c78962dda85bac29d679063b7d0d0ab1d1ab4/images/qiskit-heading.gif)

# Qiskit Bootcamp Parts III & IV: Aqua, Optimization, Machine Learning, and Chemistry

donny@ibm.com

# Gameplan

* Overview and basics
  * What is Aqua?
  * Structural Elements
  * Control flow
  * Interfaces
* Detailed component overview
* Tips and Tricks
* Learning more

But first, install Aqua:

In [0]:
!pip install qiskit-aqua

In [0]:
import warnings
warnings.filterwarnings(action='once')

# Part III: Constructing and Executing Quantum Algorithms with Aqua

# What is Aqua?

Aqua is:
* An easy to use library for running many different quantum algorithms
* A collection of reusable components which can be mixed and matched in different algorithms

Aqua is very practically driven
* We have real chemists and clients who are trying to get things done
* It is exactly what you’d expect to spring out of rapid practical need… it is very function-driven and evolves rapidly
* The guts are not so inviting in some places because it is engineered and optimized to the nines
* If you need to do something, there is a good chance someone already put it in Aqua or should
  * **Please submit feature requests!**

Aqua is built to be highly extensible and have a diversity of available interfaces:
* Declarative: Creating a QuantumAlgorithm object in python with a config dict and calling run()
* Objective: Constructing QuantumAlgorithm and component objects manually
* Aqua GUI
* Command line interface


# Plug and play components, not a circuit library

Keep in mind:

Aqua organizes reusable components into buckets, elements of which can be substituted for one another:
* Algorithms
* Variational forms
* Optimizers
* Other smaller buckets (translators, QFTs, oracles, feature maps)

It is not so much an “I give you parameters, you give me back a circuit” library, because in nearly all cases, running an algorithm is much more than executing a single circuit (e.g. VQE, q-kernel SVM, etc.)

It’s not even so much about “give me x component,” it’s more “execute this algorithm, with this set of components,” allowing you to mix and match or add your own components

There are many tests, which give good examples of execution modes and components, as well as tutorials in the qiskit-tutorials repo

Get an IDE (Pycharm is good!) and step through the code!


# Structural Elements

It is easy to see Aqua as being composed of roughly four high-level elements:
* Algorithms - the control flow and logic in generally well contained inside the algorithm class file. 
* Interfaces - Aqua is meant to be highly accessible, so it has a lot of infrascructure to allow the graphical or JSON interfaces to be almost equally accessible as coding against the library directly (no small feat!)
   * JSON definitions and declarative interfaces are dispersed throughout
   * The UI and CLI are in the top level Aqua directory
* Components - These are objects which are shared by several algorithms, as mentioned above.
* Utilities - Aqua has many useful modules and helper classes that an algorithm and application developer might want. Examples include the [operator class](https://github.com/Qiskit/aqua/blob/master/qiskit_aqua/operator.py), [random matrix generator](https://github.com/Qiskit/aqua/blob/master/qiskit_aqua/utils/random_matrix_generator.py), [cnx](https://github.com/Qiskit/aqua/blob/master/qiskit_aqua/utils/cnx.py), [run_circutis.py](https://github.com/Qiskit/aqua/blob/master/qiskit_aqua/utils/run_circuits.py), and much more.

# The Maxcut problem

Today, we're going to focus on a combinatorial optimization problem called **Maxcut**, which is solved by dividing the nodes of a weighted graph into two groups such that the egdes between the two groups carry the most possible weight. There's an [excellent notebook](https://github.com/Qiskit/qiskit-tutorial/blob/master/qiskit/aqua/optimization/maxcut_and_tsp.ipynb) 📒 by Mezzacapo et al that goes into the problem in more detail, which I highly recommend. 

The formal definition of this problem is the following:

Consider an $n$-node undirected graph *G = (V, E)* where *|V| = n* with edge weights $w_{ij}>0$, $w_{ij}=w_{ji}$, for $(i, j)\in E$. A cut is defined as a partition of the original set V into two subsets. The cost function to be optimized is in this case the sum of weights of edges connecting points in the two different subsets, *crossing* the cut. By assigning $x_i=0$ or $x_i=1$ to each node $i$, one tries to maximize the objective function (here and in the following summations run over indices 0,1,...n-1)

$$\tilde{C}(\textbf{x}) = \sum_{i,j} w_{ij} x_i (1-x_j).$$

An extension to this model has the nodes themselves carry weights. With this additional information in our model, the objective function to maximize becomes 

$$C(\textbf{x}) = \sum_{i,j} w_{ij} x_i (1-x_j)+\sum_i w_i x_i. $$

In order to find a solution to this problem on a quantum computer, one needs first to map it to an Ising Hamiltonian. This can be done with the assignment $x_i\rightarrow (1-Z_i)/2$, where $Z_i$ is the Pauli Z operator that has eigenvalues $\pm 1$. Doing this we find that 

$$C(\textbf{Z}) = \sum_{i,j} \frac{w_{ij}}{4} (1-Z_i)(1+Z_j) + \sum_i \frac{w_i}{2} (1-Z_i) = -\frac{1}{2}\left( \sum_{i<j} w_{ij} Z_i Z_j +\sum_i w_i Z_i\right)+\mathrm{const},$$

where const = $\sum_{i<j}w_{ij}/2+\sum_i w_i/2 $. In other terms, the weighted Max-Cut problem is equivalent to minimizing the Ising Hamiltonian 

$$ H = \sum_i w_i Z_i + \sum_{i<j} w_{ij} Z_iZ_j.$$

Aqua can generate the Ising Hamiltonian for the first profit function $\tilde{C}$.

### Approximate Universal Quantum Computing for Optimization Problems

There has been a considerable amount of interest in recent times about the use of quantum computers to find a solution to combinatorial problems. It is important to say that, given the classical nature of combinatorial problems, exponential speedup in using quantum computers compared to the best classical algorithms is not guaranteed. However, due to the nature and importance of the target problems, it is worth investigating heuristic approaches on a quantum computer that could indeed speed up some problem instances. Here we demonstrate an approach that is based on the Quantum Approximate Optimization Algorithm by Farhi, Goldstone, and Gutman (2014). We frame the algorithm in the context of *approximate quantum computing*, given its heuristic nature. 

The Algorithm works as follows:
1. Choose the $w_i$ and $w_{ij}$ in the target Ising problem. In principle, even higher powers of Z are allowed.
2. Choose the depth of the quantum circuit $m$. Note that the depth can be modified adaptively.
3. Choose a set of controls $\theta$ and make a trial function $|\psi(\boldsymbol\theta)\rangle$, built using a quantum circuit made of C-Phase gates and single-qubit Y rotations, parameterized by the components of $\boldsymbol\theta$. 
4. Evaluate $C(\boldsymbol\theta) = \langle\psi(\boldsymbol\theta)~|H|~\psi(\boldsymbol\theta)\rangle = \sum_i w_i \langle\psi(\boldsymbol\theta)~|Z_i|~\psi(\boldsymbol\theta)\rangle+ \sum_{i<j} w_{ij} \langle\psi(\boldsymbol\theta)~|Z_iZ_j|~\psi(\boldsymbol\theta)\rangle$ by sampling the outcome of the circuit in the Z-basis and adding the expectation values of the individual Ising terms together. In general, different control points around $\boldsymbol\theta$ have to be estimated, depending on the classical optimizer chosen. 
5. Use a classical optimizer to choose a new set of controls.
6. Continue until $C(\boldsymbol\theta)$ reaches a minimum, close enough to the solution $\boldsymbol\theta^*$.
7. Use the last $\boldsymbol\theta$ to generate a final set of samples from the distribution $|\langle z_i~|\psi(\boldsymbol\theta)\rangle|^2\;\forall i$ to obtain the answer.
    
It is our belief the difficulty of finding good heuristic algorithms will come down to the choice of an appropriate trial wavefunction. For example, one could consider a trial function whose entanglement best aligns with the target problem, or simply make the amount of entanglement a variable. In this tutorial, we will consider a simple trial function of the form

$$|\psi(\theta)\rangle  = [U_\mathrm{single}(\boldsymbol\theta) U_\mathrm{entangler}]^m |+\rangle$$

where $U_\mathrm{entangler}$ is a collection of C-Phase gates (fully entangling gates), and $U_\mathrm{single}(\theta) = \prod_{i=1}^n Y(\theta_{i})$, where $n$ is the number of qubits and $m$ is the depth of the quantum circuit. The motivation for this choice is that for these classical problems this choice allows us to search over the space of quantum states that have only real coefficients, still exploiting the entanglement to potentially converge faster to the solution.

One advantage of using this sampling method compared to adiabatic approaches is that the target Ising Hamiltonian does not have to be implemented directly on hardware, allowing this algorithm not to be limited to the connectivity of the device. Furthermore, higher-order terms in the cost function, such as $Z_iZ_jZ_k$, can also be sampled efficiently, whereas in adiabatic or annealing approaches they are generally impractical to deal with. 


References:
- A. Lucas, Frontiers in Physics 2, 5 (2014)
- E. Farhi, J. Goldstone, S. Gutmann e-print arXiv 1411.4028 (2014)
- D. Wecker, M. B. Hastings, M. Troyer Phys. Rev. A 94, 022309 (2016)
- E. Farhi, J. Goldstone, S. Gutmann, H. Neven e-print arXiv 1703.06199 (2017)

Let's begin with a weight matrix w, representing the weights between edges of the graph. To make it easier to follow along with the above mentioned notebook, we'll use the same cost matrix.

In [0]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

In [0]:
# Generating a graph of 4 nodes 

n=4 # Number of nodes in graph
G=nx.Graph()
G.add_nodes_from(np.arange(0,n,1))
elist=[(0,1,1.0),(0,2,1.0),(0,3,1.0),(1,2,1.0),(2,3,1.0)]
# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)

colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)
default_axes = plt.axes(frameon=True)
nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)

In [0]:
# Computing the weight matrix from the random graph
w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight'] 
print(w)

### Brute force approach

Try all possible $2^n$ combinations. For $n = 4$, as in this example, one deals with only 16 combinations, but for n = 1000, one has 1.071509e+30 combinations, which is impractical to deal with by using a brute force approach. 

In [0]:
best_cost_brute = 0
for b in range(2**n):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
    cost = 0
    for i in range(n):
        for j in range(n):
            cost = cost + w[i,j]*x[i]*(1-x[j])
    if best_cost_brute < cost:
        best_cost_brute = cost
        xbest_brute = x 
    print('case = ' + str(x)+ ' cost = ' + str(cost))

colors = ['r' if xbest_brute[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, pos=pos)
print('\nBest solution = ' + str(xbest_brute) + ' cost = ' + str(best_cost_brute))    

### Mapping to the Ising problem

In [0]:
qubitOp, offset = max_cut.get_max_cut_qubitops(w)
algo_input = EnergyInput(qubitOp)

### Checking that the full Hamiltonian gives the right cost 

In [0]:
#Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = ExactEigensolver(qubitOp, k=1)
result = ee.run()

algorithm_cfg = {
    'name': 'ExactEigensolver',
}

params = {
    'problem': {'name': 'ising'},
    'algorithm': algorithm_cfg
}
result = run_algorithm(params,algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

### Running it on quantum computer
We run the optimization routine using a feedback loop with a quantum computer that uses trial functions built with Y single-qubit rotations, $U_\mathrm{single}(\theta) = \prod_{i=1}^n Y(\theta_{i})$, and entangler steps $U_\mathrm{entangler}$.

In [0]:
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa, 'matrix')

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

""" declarative approach
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'SPSA',
    'max_trials': 300
}

var_form_cfg = {
    'name': 'RY',
    'depth': 5,
    'entanglement': 'linear'
}

params = {
    'problem': {'name': 'ising', 'random_seed': seed},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg,
    'backend': {'provider': 'qiskit.BasicAer', 'name': 'statevector_simulator'}
}

result = run_algorithm(params, algo_input)
"""

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

In [0]:
# run quantum algorithm with shots
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa, 'grouped_paulis')

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

"""declarative approach, update the param from the previous cell.
params['algorithm']['operator_mode'] = 'grouped_paulis'
params['backend']['provider'] = 'qiskit.BasicAer'
params['backend']['name'] = 'qasm_simulator'
params['backend']['shots'] = 1024
result = run_algorithm(params, algo_input)
"""

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))
plot_histogram(result['eigvecs'][0])

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

Now, let's try using the declarative interface to Aqua

In [0]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SPSA','max_trials': 300},
    'variational_form': {'name': 'RY','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

sol = max_cut.get_graph_solution(x)
sol

Now, let's try a new optimizer. This is a big part of the power of Aqua. The Aqua team spends time making the components broadly compatable, so experimentation is cheap and easy.

In [0]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RY','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

sol = max_cut.get_graph_solution(x)
sol

Empirical notes on optimizers:
* COBYLA is a generally good global optimizer, especially in situations when you do not have a good starting point
* SLSQP (Sequential Least Squares Programming) is very good when you have a good starting point, and the error surface can be approximated in small cavities by quadratic functions
* SPSA is generally resilient to noise, so almost always the first choise for running on the hardware or noisy simulations

Now let's try a new variational form:

In [0]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RYRZ','depth': 5,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

sol = max_cut.get_graph_solution(x)
sol

That seemed to work worse. We can also increase variational form depth:

In [0]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE','operator_mode': 'matrix'},
    'optimizer': {'name': 'SLSQP'},
    'variational_form': {'name': 'RY','depth': 8,'entanglement': 'linear'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

sol = max_cut.get_graph_solution(x)
sol

In many cases we can even swap out the **algorithm**:

In [0]:
aqua_dict = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'QAOA.Variational','operator_mode': 'matrix', 'p':2},
    'optimizer': {'name': 'COBYLA'},
    'backend': {'name': 'statevector_simulator'}
}

result = run_algorithm(aqua_dict, algo_input)

x = max_cut.sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('maxcut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

sol = max_cut.get_graph_solution(x)
sol

# Interfaces: Dictionary vs. Declaration

The form above relies on a dictionary to construct the Aqua components, and is convenient for many use cases. The dictionary allows you to experiment with low-risk changes to the algorithm in a cheap and straightforward way. 

In addition, objects can be constructed explicitly via imports and constructors. This allows you to reuse or modify components more daringly, and most algorithms developers eventully take this route for many parts of their work.

# Quick Demo: Aqua UI

_Note: The following shell command can only be run locally_

In [0]:
!qiskit_aqua_ui

# Other Neat Stuff

Aqua has many core algorithms, including:
* VQE
* QAOA
* QPE/iterativeQPE
* Hamiltonian Evolution
* Quantum-Kernel SVM
* Variational SVM
* Grover’s

# Tips and tricks (Basically the same as Terra...)

* Put many circuits into a single execution!
  * Simulators will (generally) execute these in parallel
  * Quantum Hardware does a lot of calibration for each new job, so if you send 100 jobs it will generally take 100x as long as one job with 100 circuits, even if the circuits are completely different!
* The “qasm_simulator” (cpp) backend fails gracefully over to the “qasm_simulator_py,” which is ~5x slower!
  * (ctrl-f for 'failing gracefully')
* Consider commenting out Qobj validation if you need more speed in an iterative algo, but don’t tell anyone who told you so!
* Use an IDE!! A lot of people at IBM use Pycharm. Being able to step through the code is critical!
* Look at the debug log messages. There is a ton of important info in there. See notebook here
  * Even better, save them to a file.

In [0]:
import logging
logging.getLogger('qiskit').setLevel(logging.DEBUG)

Redirecting logs to a file:

```
# Redirecting debug logs to a file (can't be done in colab):
    loggerc = logging.getLogger('qiskit_chemistry')
    loggerc.setLevel(logging.DEBUG)
    loggera = logging.getLogger('qiskit_aqua')
    loggera.setLevel(logging.DEBUG)
    loggerq = logging.getLogger('qiskit')
    loggerq.setLevel(logging.DEBUG)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    hdlr = logging.FileHandler(outdir + log_file_name, mode='w')
    hdlr.setFormatter(formatter)
    loggerc.addHandler(hdlr)
    loggera.addHandler(hdlr)
    loggerq.addHandler(hdlr)
    print('\nlog file: {}'.format(outdir + log_file_name))
# <build, execute, etc.>
# close up handlers
    loggerc.removeHandler(hdlr)
    loggera.removeHandler(hdlr)
    loggerq.removeHandler(hdlr)
    hdlr.close()
```

# Part IV: Qiskit Chemistry

# Qiskit Chemistry is a Convenient Wrapper Around Aqua and Popular Chemistry Drivers

* Qiskit Chemistry is an even more practitioner-centric tool, specifically for Chemists
* It includes two other component buckets that are not in Aqua: Chemistry drivers, and mappings (Jordan-Wigner, Bravyi-Kitaev, parity, etc.)
* The interfaces look very similar to Aqua - instantiation with a dictionary, or a standalone UI, etc.
* The core service of Qiskit Chemistry is to prepare all of the required calculations for the hamiltonian and other important observables before attempting to do Chemistry simulation or eigendecomposition in Aqua


In [0]:
!pip install qiskit-chemistry
!pip install pyscf

In [0]:
import numpy as np
import pylab
from qiskit.chemistry import QiskitChemistry

# Most Important Thing to Grasp - the Execution Flow:

* Qiskit Chemistry discovers components and reads config dictionaries
* Most fancy Chemistry package features are allowed, like orbital removal, qubit tapering, core freezing, etc.
* Calls Chemistry Drivers to calculate single and double excitation integrals
* Uses mapping to create a hamiltonian circuit out of operators for each excitation
* Calls VQE in Aqua to find eigenstates of the hamiltonian (e.g. ground state energy)
  * Also passes aux_ops (auxiliary operators) for Aqua to calculate at the end: spin, particle number, dipole moment, etc.
* Aqua creates an “evaluate_energy” function which: 
  * Takes in a scalar parameter list, creates a variational form (UCCSD, RyRz, Swap-Rz, your own, etc.) parameterized by the list
  * Executes the variational circuit + the hamiltonian circuit (or does math in the background to simulate the result quickly without sending the full circuit to the simulator)
  * Returns the resulting energy of the system
* Aqua instantiates an optimizer, and passes the energy function as the cost function to minimize
  * Use COBYLA if running on statevector simulator, SPSA if running on shot-based (qasm) because it performs well in the presence of noise
* The optimizer minimizes the energy, and returns the optimal parameters when complete
* Aqua re-creates the ground state with the optimal params, and calculates and returns the ground state energy and auxiliary operators

Let's start with a simple example, sodium hydride NaH:

In [0]:
qiskit_chemistry_dict = {
    'driver': {'name': 'PYSCF'},
    'PYSCF': {'atom': 'H .0 .0 .0; Na .0 .0 1.1', 'basis': 'sto3g'},
    'operator': {'name': 'hamiltonian', 'qubit_mapping': 'parity',
                 'two_qubit_reduction': True, 'freeze_core': True, 'orbital_reduction': [-3, -2]},
    'algorithm': {'name': 'VQE'},
    'optimizer': {'name': 'COBYLA', 'maxiter': 10000 },
    'variational_form': {'name': 'RYRZ'},
    'initial_state': {'name': 'HartreeFock'},
    'backend': {'name': 'statevector_simulator'}
}

solver = QiskitChemistry()
result = solver.run(qiskit_chemistry_dict)

In [0]:
for line in result['printable']:
    print(line)

Now, let's plot the full dissociation profile for LiH (this will take a while - maybe over 20 minutes). Notice, on line 43, that we reuse the optimal parameters from a given point as the initial parameters when we begin the following point. This gives us adiabatic-ish effects without having to run at extremely small distance steps.

Based on [this notebook](https://github.com/Qiskit/qiskit-tutorial/blob/master/qiskit/aqua/chemistry/dissociation_profile_of_molecule.ipynb) by Mezzacapo et al.

In [0]:
import numpy as np
import pylab
from qiskit.chemistry import QiskitChemistry

# Input dictionary to configure Qiskit Chemistry for the chemistry problem.
qiskit_chemistry_dict = {
    'driver': {'name': 'PYSCF'},
    'PYSCF': {'atom': '', 'basis': 'sto3g'},
    'operator': {'name': 'hamiltonian', 'qubit_mapping': 'parity',
                 'two_qubit_reduction': True, 'freeze_core': True, 'orbital_reduction': [-3, -2]},
    'algorithm': {'name': ''},
    'optimizer': {'name': 'COBYLA', 'maxiter': 10000 },
    'variational_form': {'name': 'RYRZ'},
    'initial_state': {'name': 'HartreeFock'},
    'backend': {'name': 'statevector_simulator'}
}
molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
algorithms = [{'name': 'VQE'}]

pts = np.around(np.arange(.8, 2.2, .1), decimals = 2)
energies = np.empty([len(algorithms), len(pts)])
hf_energies = np.empty(len(pts))
distances = np.empty(len(pts))
eval_counts = np.zeros([len(algorithms), len(pts)], dtype=np.intp)

print('Processing step __', end='')
for i, d in enumerate(pts):
    print('\b\b{:2d}'.format(i), end='', flush=True)
    qiskit_chemistry_dict['PYSCF']['atom'] = molecule.format(d/2) 
    for j in range(len(algorithms)):
        qiskit_chemistry_dict['algorithm'] = algorithms[j] 
        solver = QiskitChemistry()
        result = solver.run(qiskit_chemistry_dict)
        energies[j][i] = result['energy']
        hf_energies[i] = result['hf_energy']
        if algorithms[j]['name'] == 'VQE':
            eval_counts[j][i] = result['algorithm_retvals']['eval_count']
            if j == 1:
                algorithms[j]['initial_point'] = result['algorithm_retvals']['opt_params']
    distances[i] = d
print(' --- complete')

In [0]:
print('Distances: ', distances)
print('Energies:', energies)
print('Hartree-Fock energies:', hf_energies)
print('VQE num evaluations:', eval_counts)

# Tips!

* The examples above are somewhat naive, there are techniques to improve them by several orders of magnitude
* $|\text{Spin orbitals}| \approx |\text{qubits}|$ - if you increase the size of the molecule or basis, your classical execution time increases exponentially
* You don’t need to use VQE to find eigenstates! You can use QPE, QAOA, or whatever else
* Your initial point matters a lot. The way you move along a dissociation curve can dictate the fate of your resutlts.
* UCCSD is nice but it will not be runnable on Quantum hardware for a long time.
* Optimization over non-convex high-dimensional error surfaces is hard. I am sorry about that.